In [2]:
import mdptoolbox, mdptoolbox.example, mdptoolbox.util
import numpy as np


# The Cleaning Robot

Dead line : 1 st of december for both groups (note book + report describing your mdp, including a drawing)

Consider a house-cleaning robot. It can be either in the living room or at its charging station - or out  of battery. The living room can be clean or dirty. So there are five states: LD(in the living room, dirty), LC(in the living room, clean), CD(at the charger, dirty), CC(at the charger, clean), O (out of power).


####  
When in the living room,    the robot  can either choose to clean or to charge. 

When in the charging station, the robot can either choose to clean or to keep charging.

When out of power, any of the two actions (clean, charge) leads to the same results: staying out of power

####  

The reward for being  in a clean state is rc 

The reward for being in a dirty state is rd < rc at each time step

The reward for being out of power is  costcrash  -  lower or equal to rd ; let us set it equal to rd  (the living rooms becomes dirty anyway)
 

####  

When  the robot decides to  clean,
*  if it is in the living room, then it may become out of power with proba e
*  if it is in the charging station, no risk of running out of power   
*  cleaning a clean floor leaves it clean
*  cleaning a dirty floor can sometimes fail (even is battery is ok) - let eps be the probability of fail of the cleaning
     
When  the robot decides to  charge,  action charge always takes the robot to the charging station. The  dirtiness of the room can go from clean to dirty with a probability  pDust  and it stays for sure at the dirty level if dirty



#### Goal

Model the problem by a (infinite horizon) MDP (describe your model in details) 

Based on the subject, we can model the following MDP:

![Réseau MDP](https://i.imgur.com/uZZ9Rjk.jpg)

[See Original here](https://imgur.com/a/B9TAXE6)

The following function details the construction of the transition and reward matrices

In [3]:
def model(e: float, eps: float, pDust: float, rc: float, rd: float, roop:float):
    P = np.array([
        # CL
        [
            # LC LD CC CD O
            [ 1-e, 0, 0, 0, e ],         # LC
            [ 1-e-eps, eps, 0, 0, e],    # LD
            [ 1, 0, 0, 0, 0 ],           # CC
            [ 0, 1, 0, 0, 0 ],           # CD
            [ 0, 0, 0, 0, 1 ]            # O
        ],

        # CH
        [
            # LC LD CC CD O
            [ 0, 0, 0, 1, 0 ],           # LC
            [ 0, 0, 0, 1, 0 ],           # LD
            [ 0, 0, 1-pDust, pDust, 0 ], # CC
            [ 0, 0, 0, 1, 0 ],           # CD
            [ 0, 0, 0, 0, 1 ]            # O
        ]
    ])

    R = np.array([
        # CL    CH
        [ rc  , rc   ], # LC
        [ rd  , rd   ], # LD
        [ rc  , rc   ], # CC
        [ rd  , rd   ], # CD
        [ roop, roop ]  # O
    ])

    return (P, R)

Determine the best policy.

* When the probability of runing out of battery is high
* When it is low

(explain the results)

In [4]:
def print_policy(title: str, e: float, eps: float, pDust: float, rc: float, rd: float, roop:float):
    P, R = model(e, eps, pDust, rc, rd, roop)
    pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)
    # pi.setVerbose()
    pi.run()

    # print(P)
    # print(R)
    
    txt = lambda x: "CL" if x == 0 else "CH"
    eol = lambda x: "\n" if x == len(pi.policy) - 1 else " -> "
    
    print(f"[{title}]")
    print(f"Policy: ", end="")
    for i, x in enumerate(pi.policy):
        print(f"{txt(x)}", end=eol(i))

    txt = lambda x: "LC" if x == 0 else "LD" if x == 1 else "CC" if x == 2 else "CD" if x == 3 else "O"
    eol = lambda x: "\n" if x == len(pi.V) - 1 else " | "
    print(f"Value: ", end="")
    for i, x in enumerate(pi.V):
        print(f"{txt(i)}: {x}", end=eol(i))

    print()

In [5]:
e = 0.1
eps = 0.01
pDust = 0.1

rc = 1
rd = 0
ro = rd # -1

# with high probability of out of battery
print_policy("high e", 0.9, eps, pDust, rc, rd, ro)

# with low probability of out of battery
print_policy("low e", 0.1, eps, pDust, rc, rd, ro)

[high e]
Policy: CL -> CL -> CH -> CL -> CL
Value: LC: 1.0989010989010988 | LD: 0.08981936328051361 | CC: 5.301449307503799 | CD: 0.08083742695246225 | O: 0.0

[low e]
Policy: CL -> CL -> CH -> CL -> CL
Value: LC: 5.263157894736843 | LD: 4.254076159116258 | CC: 7.07673773099167 | CD: 3.828668543204633 | O: 0.0



The results here don't make much sense. It should been expected that with a higher likelyhood of running out of power, the agent would be more cautious and charge more frequently. However, the policies remain identical.

This is probably due to an error in modeling, for which a solution has not been found.

What policy is choosen if  the robot has a very good battery
(i.e. the probability of being out  of charge is very low) ? 

In [6]:
print_policy("Very low e", 0.01, eps, pDust, rc, rd, rd)

[Very low e]
Policy: CL -> CL -> CL -> CL -> CL
Value: LC: 9.174311926605506 | LD: 8.165230190984921 | CC: 9.256880733944955 | CD: 7.348707171886429 | O: 0.0



Since the threat of being out of power is slim to none, the agent will spend all its time cleaning, as it guarantees a better reward.

What policy is choosen if the  living room gets dirty   quickly 
(i.e.  pdust increases) ?  

In [10]:
print_policy("high pDust", e, eps, 0.9, rc, rd, ro)

[high pDust]
Policy: CL -> CL -> CL -> CL -> CL
Value: LC: 5.263157894736843 | LD: 4.254076159116259 | CC: 5.736842105263159 | CD: 3.828668543204633 | O: 0.0



With a higher pDust, the room will be dirty more often, this will encourage the agent to spend more time cleaning.

When e=0.1 does the best policy depend of rc ? of pDust? 

In [11]:
print_policy("Fixed e, low pDust, low rc", 0.1, eps, 0.1, 0.1, rd, rd)
print_policy("Fixed e, high pDust, low rc", 0.1, eps, 0.9, 0.1, rd, rd)
print_policy("Fixed e, low pDust, high rc", 0.1, eps, 0.1, 10, rd, rd)
print_policy("Fixed e, high pDust, high rc", 0.1, eps, 0.9, 10, rd, rd)

[Fixed e, low pDust, low rc]
Policy: CL -> CL -> CH -> CL -> CL
Value: LC: 0.5263157894736844 | LD: 0.4254076159116259 | CC: 0.7076737730991671 | CD: 0.3828668543204633 | O: 0.0

[Fixed e, high pDust, low rc]
Policy: CL -> CL -> CL -> CL -> CL
Value: LC: 0.5263157894736845 | LD: 0.42540761591162596 | CC: 0.573684210526316 | CD: 0.3828668543204633 | O: 0.0

[Fixed e, low pDust, high rc]
Policy: CL -> CL -> CH -> CL -> CL
Value: LC: 52.63157894736844 | LD: 42.540761591162585 | CC: 70.76737730991671 | CD: 38.286685432046326 | O: 0.0

[Fixed e, high pDust, high rc]
Policy: CL -> CL -> CL -> CL -> CL
Value: LC: 52.63157894736843 | LD: 42.54076159116258 | CC: 57.36842105263159 | CD: 38.286685432046326 | O: 0.0



We can tell from the following examples, that when e is fixed, the optimal policy depends on pDust.
Since pDust affects the probability that the agent will transition to a state with a lower reward (rd < rc), this makes sense. Also, being as rd is 0, changing rc whilst maintaning rd < rc will have no effect on the resulting policy.

What if costcrash  is independent of rd?

In [12]:
print_policy("Independant costcrash", 0.1, eps, pDust, rc, rd, -1)

[Independant costcrash]
Policy: CH -> CH -> CL -> CL -> CL
Value: LC: 1.0 | LD: 0.0 | CC: 1.9 | CD: 0.0 | O: -10.000000000000002



With the threat of costcrash, our agent will be much more cautious and spend more time charging. However it doesn't make much sense that it does it all at the start instead of charging at intervals.

 




Paste your python program here:



Optional: extend your program so as to take into consideration several levels of battery (high, medium, low, very low for instance)